In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [34]:
code = '005930'
url = f'https://finance.naver.com/item/sise_day.naver?code={code}'
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'
}

In [35]:
# 첫 페이지에서 페이지 링크 정보 가져오기
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')
last_page_link = soup.select('.pgRR a')
if last_page_link:
    last_page_link = last_page_link[0]['href']
    last_page = int(last_page_link.split('=')[-1])
else:
    last_page = 1

# 전체 페이지에서 날짜, 종가, 전일비, 시가, 고가, 저가, 거래량 가져오기
data = []
for page in range(1, last_page+1):
    url = f'https://finance.naver.com/item/sise_day.naver?code={code}&page={page}'
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    rows = soup.select('table > tr')
    for row in rows:
        if len(row.select('td')) == 7:
            date = row.select('td')[0].text.strip()
            close = row.select('td')[1].text.strip()
            change = row.select('td')[2].text.strip()
            open = row.select('td')[3].text.strip()
            high = row.select('td')[4].text.strip()
            low = row.select('td')[5].text.strip()
            volume = row.select('td')[6].text.strip()
            data.append([date, close, change, open, high, low, volume])

print(data)

[['2023.05.17', '65,000', '400', '65,900', '65,900', '64,800', '10,197,865'], ['2023.05.16', '65,400', '900', '65,800', '65,900', '65,300', '12,334,657'], ['2023.05.15', '64,500', '400', '64,100', '64,600', '63,900', '8,172,021'], ['2023.05.12', '64,100', '100', '63,700', '64,600', '63,600', '8,693,913'], ['2023.05.11', '64,200', '400', '64,700', '65,100', '64,200', '11,648,905'], ['2023.05.10', '64,600', '700', '65,500', '65,500', '64,300', '13,057,727'], ['2023.05.09', '65,300', '600', '65,800', '65,800', '65,100', '9,366,861'], ['2023.05.08', '65,900', '800', '66,300', '66,300', '65,400', '9,405,365'], ['2023.05.04', '65,100', '300', '65,600', '65,700', '64,700', '9,791,064'], ['2023.05.03', '65,400', '300', '65,100', '65,600', '64,900', '8,876,749'], ['2023.05.02', '65,700', '200', '66,000', '66,300', '65,100', '14,396,948'], ['2023.04.28', '65,500', '900', '65,200', '65,900', '65,000', '19,699,481'], ['2023.04.27', '64,600', '500', '64,100', '65,000', '63,300', '14,779,601'], ['20

In [37]:
import pandas as pd

# 데이터프레임 생성
columns = ['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = pd.DataFrame(data, columns=columns)

# '날짜' 열을 datetime 형식으로 변환
df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce')

# 2020년 4월 1일 이후의 데이터만 가져오기
start_date = pd.to_datetime('2020-04-01')
df = df[df['날짜'] >= start_date]

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 종목별 코드와 이름 설정
stocks = [
    ('005930', '삼성전자'),
    ('005490', 'POSCO'),
    ('005380', '현대차'),
    ('068270', '셀트리온'),
    ('032830', '삼성생명')
]

data_list = []  # 주가 데이터를 담을 리스트 생성

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'
}

for code, name in stocks:
    url = f'https://finance.naver.com/item/sise_day.naver?code={code}'
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    last_page_link = soup.select('.pgRR a')
    if last_page_link:
        last_page_link = last_page_link[0]['href']
        last_page = int(last_page_link.split('=')[-1])
    else:
        last_page = 1

    # 전체 페이지에서 날짜, 종가, 전일비, 시가, 고가, 저가, 거래량 가져오기
    data = []
    for page in range(1, last_page + 1):
        url = f'https://finance.naver.com/item/sise_day.naver?code={code}&page={page}'
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, 'html.parser')
        rows = soup.select('table > tr')
        for row in rows:
            if len(row.select('td')) == 7:
                date = row.select('td')[0].text.strip()
                close = row.select('td')[1].text.strip()
                change = row.select('td')[2].text.strip()
                opening = row.select('td')[3].text.strip()
                high = row.select('td')[4].text.strip()
                low = row.select('td')[5].text.strip()
                volume = row.select('td')[6].text.strip()
                data.append([date, close, change, opening, high, low, volume])
    data_list.append((name, data))  # 종목별 주가 데이터를 리스트에 추가

# 데이터 출력
for name, data in data_list:
    print(f"Stock Name: {name}")
    print(pd.DataFrame(data, columns=['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']))

Stock Name: 삼성전자
              날짜      종가  전일비      시가      고가      저가         거래량
0     2023.05.17  65,000  400  65,900  65,900  64,800  10,197,865
1     2023.05.16  65,400  900  65,800  65,900  65,300  12,334,657
2     2023.05.15  64,500  400  64,100  64,600  63,900   8,172,021
3     2023.05.12  64,100  100  63,700  64,600  63,600   8,693,913
4     2023.05.11  64,200  400  64,700  65,100  64,200  11,648,905
...          ...     ...  ...     ...     ...     ...         ...
6746  1996.06.27  66,900  800  67,500  67,700  66,700     155,450
6747  1996.06.26  67,700  200  67,600  67,900  66,000     136,630
6748  1996.06.25  67,500    0  66,500  68,300  65,600     112,960
6749                                                             
6750          맨앞      이전  671     672     673     674         675

[6751 rows x 7 columns]
Stock Name: POSCO
              날짜       종가    전일비       시가       고가       저가      거래량
0     2023.05.17  370,000    500  369,500  372,500  367,500  405,631
1     2023

In [24]:
import datetime

# 데이터 출력
for name, data in data_list:
    print(f"Stock Name: {name}")
    df = pd.DataFrame(data, columns=['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량'])
    df['날짜'] = pd.to_datetime(df['날짜'], format='%Y.%m.%d',  errors='coerce')
    df = df.dropna()
    df = df[df['날짜'] >= datetime.datetime(2018, 4, 1)]
    print(df)
    print()

Stock Name: 삼성전자
             날짜         종가     전일비         시가         고가         저가  \
0    2023-05-17     65,000     400     65,900     65,900     64,800   
1    2023-05-16     65,400     900     65,800     65,900     65,300   
2    2023-05-15     64,500     400     64,100     64,600     63,900   
3    2023-05-12     64,100     100     63,700     64,600     63,600   
4    2023-05-11     64,200     400     64,700     65,100     64,200   
...         ...        ...     ...        ...        ...        ...   
1259 2018-04-06  2,420,000  17,000  2,400,000  2,429,000  2,370,000   
1260 2018-04-05  2,437,000  91,000  2,370,000  2,469,000  2,367,000   
1261 2018-04-04  2,346,000  60,000  2,408,000  2,413,000  2,346,000   
1262 2018-04-03  2,406,000  21,000  2,394,000  2,407,000  2,364,000   
1263 2018-04-02  2,427,000  34,000  2,450,000  2,461,000  2,425,000   

             거래량  
0     10,197,865  
1     12,334,657  
2      8,172,021  
3      8,693,913  
4     11,648,905  
...          ...

In [39]:
df.tail()

날짜      종가    전일비      시가      고가      저가         거래량
768 2020-04-07  49,600    900  49,650  50,200  49,000  31,524,034
769 2020-04-06  48,700  1,700  47,500  48,800  47,250  23,395,726
770 2020-04-03  47,000    200  47,400  47,600  46,550  22,784,682
771 2020-04-02  46,800  1,000  46,200  46,850  45,350  21,621,076
772 2020-04-01  45,800  1,950  47,450  47,900  45,800  27,259,532

In [26]:
import pandas as pd
from datetime import datetime

columns = ['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']

# 2020년 4월 1일 이후의 데이터만 가져오기
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y.%m.%d')
df = df[df['날짜'] >= datetime(2020, 4, 1)]

# 전일 종가 칼럼을 만들어줍니다.
df['전일종가'] = df['종가'].shift(-1)

# 전일 종가 대비 상승/하락 여부를 라벨링합니다.
df['전일종가대비_상승여부'] = (df['종가'] > df['전일종가']).astype(int)

In [28]:
df = df[df['날짜'] >= datetime(2020, 4, 1)]

In [30]:
df = df.drop(['전일종가', '시가', '고가', '저가'], axis=1)

In [31]:
df

날짜      종가    전일비        거래량  전일종가대비_상승여부
0   2023-04-28  89,500    700  3,713,059            1
1   2023-04-27  88,800  1,400  2,980,163            1
2   2023-04-26  87,400  1,900  3,240,654            1
3   2023-04-25  85,500  1,700  4,096,411            0
4   2023-04-24  87,200  1,900  2,054,455            0
..         ...     ...    ...        ...          ...
757 2020-04-07  85,800  2,700  6,188,649            1
758 2020-04-06  83,100  3,500  5,784,159            1
759 2020-04-03  79,600    400  3,755,301            0
760 2020-04-02  80,000  1,600  5,394,371            1
761 2020-04-01  78,400  4,900  6,845,675            0

[762 rows x 5 columns]

In [32]:
df.to_csv('stock.csv', index=False)